In [1]:
if True:
    B = lambda x, p : bernoulli_polynomial(x, p)
    P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
    fact = lambda n : factorial(n)
else:
    B = function("B")(x,p)
    P = function("P")(x,p)
    fact = function("fact")(n)

In [2]:
def definite_integral_derivative(f, x, a, b, diff_param, hold=True):
    from sage.symbolic.integration.integral import definite_integral

    if not x.has(diff_param):
        # integration variable != differentiation variable
        ans = definite_integral(f.diff(diff_param), x, a, b, hold=hold)
    else:
        ans = SR.zero()

    if hasattr(b, 'diff'):
        if b.diff(diff_param) != 0:
            ans += f.subs(x == b) * b.diff(diff_param)
    if hasattr(a, 'diff'):
        if a.diff(diff_param) != 0:
            ans -= f.subs(x == a) * a.diff(diff_param)

    return ans

In [3]:
#sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F(x,y), x, a_x, b_x,p), y, a_y, b_y, p)
def sum_dfdx_bernoulis_euler_maclaurin_R_p(f, x, a_x, b_x, y, a_y, b_y, p):
    if logging:
        print("sum_dfdx_bernoulis_euler_maclaurin_R_p", f, x, a_x, b_x, y, a_y, b_y, p)
    dfdx_a_bernoullis = []
    dfdx_a_bernoullis_terms = []
    from sage.calculus.calculus import symbolic_sum
    from sage.symbolic.integration.integral import definite_integral
    
    for k in range(1,1+p):
        if 1 == k:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_1 = lambda y_symb, y_val, p, jx : (-1)^(p+1)*definite_integral(f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                                                      * B(x=vx,p=p)/fact(n=p),vx,0,1,hold=False) \
                                                                                .subs(y_symb == y_val)
                if logging:
                    print("k", k)
                    print("term_1(y_symb=y, y_val=a_y,jx=a_x) =", term_1(y_symb=y, y_val=a_y,jx=a_x))
                    
                R_p_1 = lambda y_symb, y_val, p : symbolic_sum(expression=term_1(y_symb=y_symb, y_val=y_val, p=p, jx=jx), \
                                                                     v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, p, jx, i=i : \
                                            term_1(y_symb=y_symb, y_val=y_val, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                R_p_1 = lambda y_symb, y_val, p : (-1)^(p+1)*definite_integral(f(x,y_symb).diff(x,p) * P(x=x,p=p)/fact(n=p), \
                                                                  x, a_x, b_x, hold=True) \
                                                                  .subs(y_symb == y_val)
                if logging:
                    print("k", k)
                    print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y, y, p))
                    print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y_symb=y, y_val=a_y, p=p))
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p_1(y_symb=y, y_val=a_y, p=p))
                print("\n")

            dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_1(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]
            if logging:
                print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
                print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

                for ii in range(len(dfdx_a_bernoullis)):
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))

        elif 2 == k:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_2 = lambda y_symb, y_val, p, jx : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                        * B(x=vx,p=p)/fact(n=p), x=vx, a=0, b=1, diff_param=y_symb, hold=False) \
                                                                                .subs(y_symb == y_val)

                if logging:
                    print("k", k)
                    print("term_2(y_symb=y, y_val=a_y,jx=a_x) =", term_2(y_symb=y, y_val=a_y, p=p, jx=a_x))

                R_p_2 = lambda y_symb, y_val, p : symbolic_sum(expression=term_2(y_symb=y_symb, y_val=y_val, p=p, jx=jx), \
                                                                 v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, p, jx, i=i : \
                                            term_2(y_symb=y_symb, y_val=y_val, p=p, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                R_p_2 = lambda y_symb, y_val, p : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                            * P(x=x,p=p)/fact(n=p), x=x, a=a_x, b=b_x, diff_param=y_symb) \
                                                  .subs(y_symb == y_val)

                if logging:
                    print("k", k)
                    print("R_p_2(y_symb=y, y_val=a_y) =", R_p_2(y, y, p))
                    print("R_p_2(y_symb=y, y_val=a_y) =", R_p_2(y_symb=y, y_val=a_y, p=p))
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p_2(y_symb=y, y_val=a_y, p=p))
                print("\n")
        
            dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_2(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]
            if logging:
                print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
                print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

                for ii in range(len(dfdx_a_bernoullis)):
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))
        else:
            if use_int_R_p_j:
                vx, jx = var("v_x, j_x")
                term_3 = lambda y_symb, y_val, p, jx : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                                                 .subs(x == vx + jx) \
                                        * B(x=vx,p=p)/fact(n=p), x=vx, a=0, b=1, diff_param=y_symb, hold=False) \
                                            .diff(y_symb, k-2) .subs(y_symb == y_val)

                if logging:
                    print("k", k)
                    print("term_3(y_symb=y, y_val=a_y,jx=a_x) =", term_3(y_symb=y, y_val=a_y, p=p, jx=a_x))
                    
                R_p_3 = lambda y_symb, y_val, p : symbolic_sum(expression=term_3(y_symb=y_symb, y_val=y_val, p=p, jx=jx), \
                                                                 v=jx, a=a_x, b=b_x-1)
                dfdx_a_bernoullis_terms += [lambda y_symb, y_val, p, jx, i=i : \
                                            term_3(y_symb=y_symb, y_val=y_val, p=p, jx=jx)*(bernoulli(k)/factorial(k))]
            else:
                R_p_3 = lambda y_symb, y_val, p : (-1)^(p+1)*definite_integral_derivative(f=f(x,y_symb).diff(x,p) \
                                                    * P(x=x,p=p)/fact(n=p), x=x, a=a_x, b=b_x, diff_param=y_symb) \
                                                    .diff(y_symb, k-2).subs(y_symb == y_val)
                
            if logging:
                print("R_p(y_symb=y, y_val=a_y) =", R_p_3(y_symb=y, y_val=a_y, p=p))
                print("\n")

            dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_3(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]
            if logging:
                print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
                print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))

                for ii in range(len(dfdx_a_bernoullis)):
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
                    print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
                    print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))

    if logging:
        print("dfdx_a_bernoullis =", dfdx_a_bernoullis)
        print("len(dfdx_a_bernoullis) =", len(dfdx_a_bernoullis))
        
        for ii in range(len(dfdx_a_bernoullis)):
            print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii])
            print("dfdx_a_bernoullis [",ii, "]()=", dfdx_a_bernoullis[ii](y, y, p))
            print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y, a_y, p))
            print("dfdx_a_bernoullis [",ii, "](a_y)=", dfdx_a_bernoullis[ii](y_symb=y, y_val=a_y, p=p))



    sum_dfdx_a_bernoullis = lambda y_symb, y_val, p : sum([dfdx_a_bernoullis[i](y_symb, y_val, p) \
                                                      for i in range(len(dfdx_a_bernoullis))])
    
    sum_dfdx_a_bernoullis_terms = lambda y_symb, y_val, p, jx : sum([dfdx_a_bernoullis_terms[i](y_symb, y_val, p, jx) \
                                                               for i in range(len(dfdx_a_bernoullis_terms))])
    
    if logging:
        print("sum_dfdx_a_bernoullis =", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a_y) =", sum_dfdx_a_bernoullis(y_symb=y, y_val=a_y, p=p))
    
    s = - sum_dfdx_a_bernoullis(y_symb=y, y_val=a_y, p=p)
    if b_y is Infinity:
        s_terms = lambda jx : - sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=a_y, p=p, jx=jx)
    else:
        s += sum_dfdx_a_bernoullis(y_symb=y, y_val=b_y, p=p)
        s_terms = lambda jx : sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=b_y, p=p, jx=jx) - sum_dfdx_a_bernoullis_terms(y_symb=y, y_val=a_y, p=p, jx=jx)
    return s, s_terms

In [4]:
from IPython.display import display, Math, Latex

In [5]:
a_x = var("n_x")
b_x = var("b_x")
a_y = var("n_y")
b_y = var("b_y")

assume(a_x, 'real')
assume(b_x, 'real')
assume(a_y, 'real')
assume(b_y, 'real')

assume(a_x, 'integer')
assume(b_x, 'integer')
assume(a_y, 'integer')
assume(b_y, 'integer')

assume(a_x > 0)
assume(b_x > 0)
assume(a_y > 0)
assume(b_y > 0)

x,y = var("x,y")

n = QQ['n'].0
j = QQ['j'].0

p = 4
p = 3
#p = 2
#p = 1

F = function("F")(x, y)
print(F)

s [_SAGE_VAR_n_x>0]
parser_output [n_x > 0]
s [_SAGE_VAR_b_x>0]
parser_output [b_x > 0]
s [_SAGE_VAR_n_y>0]
parser_output [n_y > 0]
s [_SAGE_VAR_b_y>0]
parser_output [b_y > 0]
F(x, y)


In [6]:

use_int_R_p_j=False
logging = True
sum_R_p = sum_dfdx_bernoulis_euler_maclaurin_R_p(F(x,y), x, a_x, b_x,    y, a_y, b_y, p)

if type(sum_R_p) is tuple:
    print("sum_R_p=",sum_R_p[0])
    print("sum_R_p=$",latex(sum_R_p[0]),"$")
    display(Math(latex(sum_R_p[0])))
else:
    print("sum_R_p=",sum_R_p)
    print("sum_R_p=$",latex(sum_R_p),"$")
    display(Math(latex(sum_R_p)))

sum_dfdx_bernoulis_euler_maclaurin_R_p F(x, y) x n_x b_x y n_y b_y 3
k 1
R_p_1(y_symb=y, y_val=a_y) = integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x), x, n_x, b_x)
R_p_1(y_symb=y, y_val=a_y) = 1/12*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x)
R_p(y_symb=y, y_val=a_y) = 1/12*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x)


dfdx_a_bernoullis = [<function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd745af0>]
len(dfdx_a_bernoullis) = 1
dfdx_a_bernoullis [ 0 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd745af0>
dfdx_a_bernoullis [ 0 ]()= -1/2*integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x), x, n_x, b_x)
dfdx_a_bernoullis [ 0 ](a_y)= -1/24*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x

<ipython-input-3-4112e1ee9c8c>:25: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y, y, p))
<ipython-input-3-4112e1ee9c8c>:26: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  print("R_p_1(y_symb=y, y_val=a_y) =", R_p_1(y_symb=y, y_val=a_y, p=p))
<ipython-input-3-4112e1ee9c8c>:29: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
 

](a_y)= 1/144*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x)
dfdx_a_bernoullis [ 0 ](a_y)= 1/144*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x), x, n_x, b_x)
dfdx_a_bernoullis [ 1 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd7561f0>
dfdx_a_bernoullis [ 1 ]()= 1/12*integrate(1/12*(2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, y), x, x, x, y), x, n_x, b_x)
dfdx_a_bernoullis [ 1 ](a_y)= 1/144*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x, n_y), x, n_x, b_x)
dfdx_a_bernoullis [ 1 ](a_y)= 1/144*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x, n_y), x, n_x, b_x)


<ipython-input-3-4112e1ee9c8c>:66: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_2(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]
<ipython-input-3-4112e1ee9c8c>:91: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  print("R_p(y_symb=y, y_val=a_y) =", R_p_3(y_symb=y, y_val=a_y, p=p))


R_p(y_symb=y, y_val=a_y) = 1/12*integrate((2*(x - floor(x))^3 - 3*(x - floor(x))^2 + x - floor(x))*diff(F(x, n_y), x, x, x, n_y, n_y), x, n_x, b_x)


dfdx_a_bernoullis = [<function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd745af0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd7561f0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd5d5550>]
len(dfdx_a_bernoullis) = 3
dfdx_a_bernoullis [ 0 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd745af0>
dfdx_a_bernoullis [ 0 ]()= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 1 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd7561f0>
dfdx_a_bernoullis [ 1 ]()= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 2 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd5d5550>
dfdx_a_bernoulli

<ipython-input-3-4112e1ee9c8c>:94: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  dfdx_a_bernoullis += [lambda y_symb, y_val, p, i=i : R_p_3(y_symb=y_symb, y_val=y_val, p=p)*(bernoulli(k)/factorial(k))]


dfdx_a_bernoullis [ 2 ](a_y)= 0
dfdx_a_bernoullis [ 2 ](a_y)= 0
dfdx_a_bernoullis = [<function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd745af0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd7561f0>, <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd5d5550>]
len(dfdx_a_bernoullis) = 3
dfdx_a_bernoullis [ 0 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd745af0>
dfdx_a_bernoullis [ 0 ]()= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 0 ](a_y)= 0
dfdx_a_bernoullis [ 1 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd7561f0>
dfdx_a_bernoullis [ 1 ]()= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 1 ](a_y)= 0
dfdx_a_bernoullis [ 2 ]()= <function sum_dfdx_bernoulis_euler_maclaurin_R_p.<locals>.<lambda> at 0x7fcbcd5d5550>
dfdx_a_bernoullis [ 2 ]()= 0
dfdx_a_bernoullis [ 2 ](a_y)= 0
dfdx_a_bernoullis [ 2 ](a_y)= 0
sum_dfdx_

<IPython.core.display.Math object>